<a href="https://colab.research.google.com/github/deebee08/Twitter_Data_Harvester/blob/main/AI_Chatbot_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tweepy openai


In [ ]:
import tweepy
import openai

# Twitter API credentials
consumer_key = 'my_consumer_key'
consumer_secret = 'my_consumer_secret'
access_token = 'my_access_token'
access_token_secret = 'my_access_token_secret'
#keys replaced for privacy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# OpenAI API key
openai.api_key = "my_openai_api_key"


In [ ]:
def generate_response_prompt(tweet_text):
    return f"""
    The following tweet reports a bird collision or asks for advice about bird conservation:

    Tweet: "{tweet_text}"

    Provide a helpful, concise response that guides the user on how to handle the situation or prevent future collisions. Refer to https://flap.org/stop-birds-from-hitting-windows/ and avoid random or speculative information. Be supportive and factual.
    """


In [ ]:
def generate_response(tweet_text):
    prompt = generate_response_prompt(tweet_text)
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=prompt,
        max_tokens=150,
        temperature=0.5,
        top_p=0.9
    )
    return response.choices[0].text.strip()


In [ ]:
def respond_to_retweets():
    for tweet in tweepy.Cursor(api.user_timeline, screen_name="birdcrash_bot", tweet_mode='extended').items():
        if 'RT @' in tweet.full_text:  # only respond to retweets
            response_text = generate_response(tweet.full_text)
            try:
                api.update_status(status=response_text, in_reply_to_status_id=tweet.id, auto_populate_reply_metadata=True)
                print(f"Responded to tweet {tweet.id}")
            except Exception as e:
                print(f"Error responding to tweet {tweet.id}: {e}")


In [ ]:
import time

while True:
    respond_to_retweets()
    time.sleep(300)  # Check every 5 minutes
